In [31]:
%matplotlib inline

from path import Path

import math
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import statsmodels.formula.api as sm
from statsmodels.tsa import tsatools, stattools
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics import tsaplots

In [32]:
to = pd.read_csv('ToysRUsRevenues.csv')

In [33]:
to

,Index,QuarterYear,Revenue(in million $),Quarter
0,1,Q1-92,1026,Q1
1,2,Q2-92,1056,Q2
2,3,Q3-92,1182,Q3
3,4,Q4-92,2861,Q4
4,5,Q1-93,1172,Q1
5,6,Q2-93,1249,Q2
6,7,Q3-93,1346,Q3
7,8,Q4-93,3402,Q4
8,9,Q1-94,1286,Q1
9,10,Q2-94,1317,Q2


In [34]:
to.dtypes

Index                     int64
QuarterYear              object
Revenue(in million $)     int64
Quarter                  object
dtype: object

In [35]:
date = to.QuarterYear.str.replace(r'(Q\d)-(\d+)', r'19\2-\1')
to['Date'] = pd.to_datetime(date).dt.strftime('%Q-%Y')


In [36]:
to

,Index,QuarterYear,Revenue(in million $),Quarter,Date
0,1,Q1-92,1026,Q1,1992-01-01 00:00:00
1,2,Q2-92,1056,Q2,1992-04-01 00:00:00
2,3,Q3-92,1182,Q3,1992-07-01 00:00:00
3,4,Q4-92,2861,Q4,1992-10-01 00:00:00
4,5,Q1-93,1172,Q1,1993-01-01 00:00:00
5,6,Q2-93,1249,Q2,1993-04-01 00:00:00
6,7,Q3-93,1346,Q3,1993-07-01 00:00:00
7,8,Q4-93,3402,Q4,1993-10-01 00:00:00
8,9,Q1-94,1286,Q1,1994-01-01 00:00:00
9,10,Q2-94,1317,Q2,1994-04-01 00:00:00


In [37]:
#convert to time series
revenue_ts = pd.Series(to['Revenue(in million $)'].values, index=to.Date, name = 'Revenue')
revenue_ts.index = pd.DatetimeIndex(revenue_ts.index)
revenue_df = tsatools.add_trend(revenue_ts, trend = 'ct')
revenue_df['Quarter'] = revenue_df.index.quarter

In [38]:
nValid = 2
nTrain = len(revenue_ts) - nValid

#partition the data
train_df = revenue_df[:nTrain]
valid_df = revenue_df[nTrain:]

revenue_lm_trendseason = sm.ols(formula = 'Revenue ~ trend + C(Quarter)', data = train_df).fit()

#fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(9, 7.5))

#revenue_lm_trendseason.predict(train_df).plot(ax=axes[0], color='C1')
#revenue_lm_trendseason.predict(valid_df).plot(ax=axes[0], color='C1', linestyle='dashed')
    
#residual = train_df.Revenue - revenue_lm_trendseason.predict(train_df)
#residual.plot(ax=axes[1], color='C1')
#residual = valid_df.Revenue - revenue_lm_trendseason.predict(valid_df)
#residual.plot(ax=axes[1], color='C1', linestyle='dashed')

#graphLayout(axes, train_df, valid_df)

#plt.show()
print(revenue_lm_trendseason.summary())

                            OLS Regression Results                            
Dep. Variable:                Revenue   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.967
Method:                 Least Squares   F-statistic:                     97.18
Date:                Sat, 13 Jun 2020   Prob (F-statistic):           2.13e-07
Time:                        18:10:01   Log-Likelihood:                -88.547
No. Observations:                  14   AIC:                             187.1
Df Residuals:                       9   BIC:                             190.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         906.7500    115.346     

C:\Users\GETHSEMAN\anaconda3\lib\site-packages\scipy\stats\stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


In [39]:
revenue_df

,Revenue,const,trend,Quarter
Date,,,,
1992-01-01,1026,1.0,1.0,1
1992-04-01,1056,1.0,2.0,2
1992-07-01,1182,1.0,3.0,3
1992-10-01,2861,1.0,4.0,4
1993-01-01,1172,1.0,5.0,1
1993-04-01,1249,1.0,6.0,2
1993-07-01,1346,1.0,7.0,3
1993-10-01,3402,1.0,8.0,4
1994-01-01,1286,1.0,9.0,1
